In [ ]:
soln = '''
import numpy as np
from typing import Tuple

def control_one_goal(position: np.ndarray,
                     goal: np.ndarray,
                     orientation: float) -> Tuple[float, float]:
  
  direction_to_goal = np.arctan2(goal[1] - position[1], goal[0] - position[0])
  difference_in_orientation = direction_to_goal - orientation
  normalized_difference = (difference_in_orientation + np.pi) % (2 * np.pi) - np.pi
  
  torque = np.clip(-2. * normalized_difference, -1., 1.)  # Sign flipped here for correct orientation
  
  distance_to_goal = np.linalg.norm(goal - position)
  if distance_to_goal < 0.5:
    thrust = 0.0
  else:
    thrust = min(distance_to_goal / 10., 1.)
  
  return torque, thrust
'''

In [1]:
import dataclasses

import numpy as np

import chat
import llm_task
import enact
import demo_utils
import asteroids_2goal as asteroids


In [ ]:
store = enact.Store(backend=enact.FileBackend('/home/max/Documents/enact/examples/store_backend/'))
digest_str = 

with store:
  reference = enact.Ref.from_id(digest_str)
  soln_string = reference().response().output()

print(soln_string)

In [ ]:
import dataclasses

import numpy as np

import chat
import llm_task
import enact
import demo_utils
import asteroids_2goal as asteroids


task_prompt = '''
Write a concise python function with the following signature.
Answer with a single code block.'''
code_gen_prompt = '''

```
def control_one_goal(position: np.ndarray,
                     goal: np.ndarray,
                     orientation: float) -> Tuple[float, float]
  """Control an asteroids-style ship to move to a goal.
  
  Args:
    position: A 2D vector representing the position of the ship.
    goal: A 2D vector representing the position of the goal.
    orientation: A float representing the orientation of the ship in radians.
  Returns:
    A pair of floats representing:
      torque: The amount of torque to apply to the ship, between -1 and 1.
      thrust: The amount of thrust to apply to the ship, between 0.0 and 1
  """
  <INSERT IMPLEMENTATION HERE>
```
'''

code_gen = llm_task.Task(
  task_prompt=task_prompt,
  chat_agent=chat.GPTAgent(model='gpt-4'))
code_gen.add_example(
  '''```def add(x: int, y: int):\n  <INSERT IMPLEMENTATION HERE>```''',
  '''```python\ndef add(x: int, y: int):\n  return x + y\n```''')

@enact.typed_invokable(enact.NoneResource, enact.Str)
@dataclasses.dataclass
class CreatePolicy(enact.Invokable):
  code_gen: llm_task.Task
  
  def call(self):
    return self.code_gen(enact.Str(code_gen_prompt))


sample_input = (np.zeros((2,)), np.zeros((2,)), 0.0)

PolicyChecker = demo_utils.get_policy_checker(asteroids.Action,
                                              asteroids.create_trajectory,
                                              asteroids.plot_trajectory,
                                              sample_input,
                                              func_name='control_one_goal')

store = enact.Store(backend=enact.FileBackend('/home/max/Documents/enact/examples/store_backend/'))
store = enact.Store()
with store:
  post_processor_ref = enact.commit(PolicyChecker())
  code_gen.post_processor = post_processor_ref
  code_gen.max_retries = 2
  create_policy = CreatePolicy(code_gen)
